<a href="https://colab.research.google.com/github/GeoKauko/TheNavySeals/blob/main/1_Preprocessing_integrated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries

## Import libraries
import os
from osgeo import gdal

import rasterio
from rasterio import windows
from rasterio.windows import Window

from rasterio.plot import reshape_as_image
import rasterio.mask
from rasterio.features import rasterize

import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping, Point, box, Polygon
from shapely.ops import cascaded_union

import numpy as np

from rasterio.features import geometry_mask
from shapely.geometry import box
from rasterio.features import geometry_mask


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

input_path = '/content/drive/My Drive/data/input'
output_path = '/content/drive/My Drive/data/output'

pansharpened_path = '/content/drive/My Drive/data/input/pansharpened'
pansharpened_reduced_path = '/content/drive/My Drive/data/input/pansharpened_reduced'


In [ ]:
## Resize pansharpened images to 8-bit

# Function to reduce pixel size from 11 bits to 8 bits for a single image
def reduce_pixel_size(input_path, output_path):
    # Ensure the output directory exists
    output_dir = os.path.dirname(output_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with rasterio.open(input_path) as src:
        # Read the image as a numpy array
        image_array = src.read(1, masked=True)

        # Rescale the pixel values to fit within 8-bit range (0-255)
        scaled_array = (image_array / (2**11 - 1) * 255).astype(np.uint8)

        # Create a new raster profile with 8-bit pixel depth
        profile = src.profile
        profile.update(dtype=rasterio.uint8)

        # Write the scaled array to a new raster file
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(scaled_array, 1)

In [ ]:
## Merge pansharpened images

def merge_rasters(input_folder, output_file):
    # List to hold the file paths of the rasters to be merged
    input_files = []

    # Loop through the folder and add all .tif files to the list
    for file_name in os.listdir(input_folder):
        print(file_name)
        if file_name.endswith('.TIF'):
            input_files.append(os.path.join(input_folder, file_name))

    # Check if we have any input files
    if not input_files:
        raise FileNotFoundError("No .tif files found in the specified folder.")

    # Open the input files
    src_files_to_mosaic = []
    for file in input_files:
        src = gdal.Open(file)
        if src:
            src_files_to_mosaic.append(src)
        else:
            print(f"Failed to open {file}")

    # Create a virtual raster from the input files
    vrt = gdal.BuildVRT('temporary.vrt', src_files_to_mosaic)

    # Write the virtual raster to a new file
    gdal.Translate(output_file, vrt)

    # Cleanup
    vrt = None
    for src in src_files_to_mosaic:
        src = None

    print(f"Merged raster saved as {output_file}")

In [ ]:
## Crop to 224x224 px

def split_and_save_image(dataset, part_width, part_height, output_dir):
    # Get image dimensions
    width = dataset.RasterXSize
    height = dataset.RasterYSize

    # Calculate the number of parts
    num_parts_x = width // part_width
    num_parts_y = height // part_height

    # Get the number of bands
    bands = dataset.RasterCount

    # Split the image and save
    for i in range(num_parts_x):
        for j in range(num_parts_y):
            x_offset = i * part_width
            y_offset = j * part_height

            # Read the split region
            part = dataset.ReadAsArray(x_offset, y_offset, part_width, part_height)

            # Expand dimensions if there's only one band
            if bands == 1:
               part = np.expand_dims(part, axis=0)

            # Create a new GDAL dataset to save the split part
            driver = gdal.GetDriverByName('GTiff')
            output_path = os.path.join(output_dir, f'part_{i}_{j}.tif')
            out_dataset = driver.Create(output_path, part_width, part_height, bands, gdal.GDT_UInt16)

            # Write data to the new dataset
            for band in range(bands):
                out_band = out_dataset.GetRasterBand(band + 1)
                out_band.WriteArray(part[band])

            # Set georeference and projection
            geo_transform = list(dataset.GetGeoTransform())
            geo_transform[0] += x_offset * geo_transform[1]
            geo_transform[3] += y_offset * geo_transform[5]
            out_dataset.SetGeoTransform(tuple(geo_transform))
            out_dataset.SetProjection(dataset.GetProjection())

            # Save and close
            out_dataset.FlushCache()
            del out_dataset

    # Close the original dataset
    del dataset

In [ ]:
## Mask raster

def create_polygon_from_pixels(row, col, transform):
    """
    Create a polygon from a center pixel (row, col) and its 24 surrounding pixels (5x5 block).
    """
    # Calculate the coordinates of the top-left corner of the top-left pixel
    top_left_x = transform[0] + (col - 2) * transform[1] + (row - 2) * transform[2]
    top_left_y = transform[3] + (col - 2) * transform[4] + (row - 2) * transform[5]

    # Pixel dimensions
    pixel_width = abs(transform[1])
    pixel_height = abs(transform[5])

    # Calculate the coordinates for the 5x5 block of pixels
    polygon_coords = [
        (top_left_x, top_left_y),
        (top_left_x + 5 * pixel_width, top_left_y),
        (top_left_x + 5 * pixel_width, top_left_y - 5 * pixel_height),
        (top_left_x, top_left_y - 5 * pixel_height),
        (top_left_x, top_left_y)
    ]

    return Polygon(polygon_coords)

def raster_points_to_polygons(raster_path, shape_path):
    # Read raster data
    raster_dataset = gdal.Open(raster_path)
    raster_geotransform = raster_dataset.GetGeoTransform()

    # Read shapefile
    shapefile_gdf = gpd.read_file(shape_path)
    shape_crs = shapefile_gdf.crs

    polygons = []
    for point in shapefile_gdf.geometry:
        # Convert point coordinates to raster coordinates
        x, y = point.x, point.y
        col = int((x - raster_geotransform[0]) / raster_geotransform[1])
        row = int((y - raster_geotransform[3]) / raster_geotransform[5])

        # Create polygon around the pixel and its 24 surrounding pixels
        polygon = create_polygon_from_pixels(row, col, raster_geotransform)
        polygons.append(polygon)

    result_gdf = gpd.GeoDataFrame(geometry=polygons, crs=shape_crs)

    return result_gdf


def mask_raster_with_polygon(input_raster_path, shapefile_path, output_raster_path, value=1):
    """
    Create a copy of a raster, set all its values to 0, overlay it with a polygon shapefile,
    and set all pixels underneath polygons to a specified value.

    Args:
    - input_raster_path (str): Path to the input raster.
    - shapefile_path (str): Path to the shapefile containing polygons.
    - output_raster_path (str): Path to save the masked raster.
    - value (int, optional): Value to set for pixels underneath polygons. Defaults to 1.
    """
    # Open the input raster for reading
    with rasterio.open(input_raster_path) as src:
        # Read raster data
        raster_data = src.read(1)
        # Get metadata
        meta = src.meta

    # Set all values to 0
    raster_data.fill(0)

    # Read the shapefile containing polygons
    polygons = gpd.read_file(shapefile_path)

    # Create mask from polygons
    mask = geometry_mask(polygons.geometry, out_shape=raster_data.shape, transform=src.transform, invert=True)

    # Set pixels underneath polygons to the specified value
    raster_data[mask] = value

    # Save the masked raster
    with rasterio.open(output_raster_path, 'w', **meta) as dst:
        dst.write(raster_data, 1)


In [ ]:
## Crop image & mask -> count seals & make csv with seal distribution

In [ ]:
## Split into validation, test and training data sets